In [2]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer, ActivationCache, utils, patching
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output

pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

from haystack_utils import get_mlp_activations
import haystack_utils

%reload_ext autoreload
%autoreload 2

In [5]:
model = HookedTransformer.from_pretrained("EleutherAI/pythia-70m",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device)

german_data = haystack_utils.load_json_data("data/german_europarl.json")[:200]
english_data = haystack_utils.load_json_data("data/english_europarl.json")[:200]


english_activations = {}
german_activations = {}
for layer in range(3, 4):
    english_activations[layer] = get_mlp_activations(english_data, layer, model, mean=False)
    german_activations[layer] = get_mlp_activations(german_data, layer, model, mean=False)

LAYER_TO_ABLATE = 3
NEURONS_TO_ABLATE = [669]
MEAN_ACTIVATION_ACTIVE = german_activations[LAYER_TO_ABLATE][:, NEURONS_TO_ABLATE].mean()
MEAN_ACTIVATION_INACTIVE = english_activations[LAYER_TO_ABLATE][:, NEURONS_TO_ABLATE].mean()

def deactivate_neurons_hook(value, hook):
    value[:, :, NEURONS_TO_ABLATE] = MEAN_ACTIVATION_INACTIVE
    return value
deactivate_neurons_fwd_hooks=[(f'blocks.{LAYER_TO_ABLATE}.mlp.hook_post', deactivate_neurons_hook)]

def activate_neurons_hook(value, hook):
    value[:, :, NEURONS_TO_ABLATE] = MEAN_ACTIVATION_ACTIVE
    return value
activate_neurons_fwd_hooks=[(f'blocks.{LAYER_TO_ABLATE}.mlp.hook_post', activate_neurons_hook)]

all_ignore, not_ignore = haystack_utils.get_weird_tokens(model, plot_norms=False)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
data/german_europarl.json: Loaded 2000 examples with 152 to 2000 characters each.
data/english_europarl.json: Loaded 2000 examples with 165 to 2000 characters each.


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [12]:
def get_attention_pattern_activations(cache: ActivationCache, layer: int, n_pos=-1):
    block_name = f'blocks.{layer}.attn.hook_pattern'
    original_activations = cache[block_name].mean(0)

    if n_pos == -1:
        n_pos = original_activations.shape[1]

    for i, pattern in enumerate(original_activations):
        haystack_utils.imshow(pattern[:n_pos, :n_pos].cpu(), title=f"L{layer}H{i} Attention patterns", width=700)


tokens = model.to_tokens(german_data[:20])
original_loss, original_cache = model.run_with_cache(tokens, return_type="loss")

get_attention_pattern_activations(original_cache, layer=0, n_pos=20)
get_attention_pattern_activations(original_cache, layer=1, n_pos=20)
get_attention_pattern_activations(original_cache, layer=2, n_pos=20)
# get_attention_pattern_activations(original_cache, layer=3, n_pos=20)
# get_attention_pattern_activations(original_cache, layer=4, n_pos=20)
# get_attention_pattern_activations(original_cache, layer=5, n_pos=20)
# get_attention_pattern_activations(original_cache, layer=6, n_pos=20)

## Trigram circuit

Previous token head could copy over skip-bigram information, or copy over what the previous token is so the current token can compute trigram information using a trigram table, perhaps learned in the embed, of previous tokens -> current token -> most common next token.

Test:
- If head has a positive direct effect on the output it's a skip bigram. 
- If not, generate a corrupted head cache and patch components into a clean run to see what's reading from it.

We think it's doing trigrams rather than skip bigrams so we could potentially save some time by testing for skip trigrams immediately.

Trigram: each token embedding has a table of possible previous token x implies next token z. L0H7 copies across previous token x and multiplies it with the trigram take to get next token z. The trigram table must be read in from the current position token embedding so it's not happening in the previous token head which only reads from the previous token. It either happens in a future head that only reads from the current token

Hypothesis: previous token head L0H7 is responsible for most of the trigram loss increase. Two options for how it's used:
- a global bigram table in the embedding or in a later component contains the global bigram completions that sometimes include completions which only make sense in German. The attention head copies this information over 
- an MLP layer contains German-specific bigram completions. The bigram completions are combined with the previous token and selected when the prompt is in German, most likely by direct reference to the German context neuron.

To look for the MLP layer, get a cache where L2H1 has been zero ablated or otherwise distorted, and then patch MLP components from this cache into a clean run and see which one damages the loss.

In [27]:
# Get top common german tokens excluding punctuation
token_counts = torch.zeros(model.cfg.d_vocab).cuda()
for example in tqdm(german_data):
    tokens = model.to_tokens(example)
    for token in tokens[0]:
        token_counts[token.item()] += 1

punctuation = ["\n", ".", ",", "!", "?", ";", ":", "-", "(", ")", "[", "]", "{", "}", "<", ">", "/", "\\", "\"", "'"]
leading_space_punctuation = [" " + char for char in punctuation]
punctuation_tokens = model.to_tokens(punctuation + leading_space_punctuation + [' –', " ", '  ', "<|endoftext|>"])[:, 1].flatten()
token_counts[punctuation_tokens] = 0
token_counts[all_ignore] = 0

top_counts, top_tokens = torch.topk(token_counts, 100)
print(model.to_str_tokens(top_tokens[:100]))

def get_random_selection(tensor, n=12):
    # Hacky replacement for np.random.choice
    return tensor[torch.randperm(len(tensor))[:n]]

def generate_random_prompts(end_string, n=50, length=12):
    # Generate a batch of random prompts ending with a specific ngram
    end_tokens = model.to_tokens(end_string).flatten()[1:]
    prompts = []
    for i in range(n):
        prompt = get_random_selection(top_tokens[:max(50, length)], n=length).cuda()
        prompt = torch.cat([prompt, end_tokens])
        prompts.append(prompt)
    prompts = torch.stack(prompts)
    return prompts

prompts = generate_random_prompts(" Vorschlägen", n=100, length=20)

  0%|          | 0/200 [00:00<?, ?it/s]

[' der', 'en', ' die', ' und', 'ung', 'ä', ' in', ' den', ' des', ' zu', 'ch', 'n', 'st', 're', 'z', ' von', ' für', 'äsident', ' Pr', 'ischen', 't', 'ü', 'icht', 'gen', ' ist', ' auf', ' dass', 'ge', 'ig', ' im', 'in', ' über', 'g', ' das', 'te', ' er', 'men', ' w', 'es', ' an', 'ß', ' wir', ' eine', 'f', ' W', 'hen', 'w', ' Europ', ' ich', 'ungen', 'ren', 'le', ' dem', 'ten', ' ein', 'e', ' Z', ' Ver', 'der', ' B', ' mit', ' dies', 'h', ' nicht', 'ungs', 's', ' G', ' z', 'it', ' Herr', ' es', 'l', ' S', 'ich', 'lich', ' An', 'heit', 'ie', ' Er', ' zur', ' V', ' ver', 'u', 'hr', 'chaft', 'Der', ' Ich', ' Ab', ' haben', 'i', 'ant', 'chte', ' mö', 'er', ' K', 'igen', ' Ber', 'ür', ' Fra', 'em']


In [31]:
# see if zero ablating the clean prev token head increases loss
# can't mean ablate the pattern because it always looks at previous tokens
# could mean ablate the output to it passes forwards gibberish as the previous token
common_tokens = haystack_utils.get_common_tokens(german_data, model, all_ignore, k=50)
random_prompts = haystack_utils.generate_random_prompts(" Vorschlägen", model, common_tokens, 100, length=20)[:, :-4]
model.set_use_attn_result(True)

l0_heads = 'blocks.0.attn.hook_result'
_, original_cache = model.run_with_cache(random_prompts, return_type="loss", loss_per_token=True)
original_loss, _ = model.run_with_cache(prompts, return_type="loss", loss_per_token=True)

mean_l0h7_out = original_cache[l0_heads][:, :, 7, :].mean((0, 1))
def ablate_l2h7_hook(value, hook):
    value[:, -2, 7, :] = mean_l0h7_out
    return value

with model.hooks([(l0_heads, ablate_l2h7_hook)]):
    ablated_loss, ablated_cache = model.run_with_cache(prompts, return_type="loss", loss_per_token=True)

print((ablated_loss[:, -1] - original_loss[:, -1]).mean())

  0%|          | 0/200 [00:00<?, ?it/s]

tensor(3.0581, device='cuda:0')


In [47]:
original_loss, original_cache = model.run_with_cache(prompts, return_type="loss", loss_per_token=True)
with model.hooks([(l0_heads, ablate_l2h7_hook)]):
    ablated_loss, ablated_cache = model.run_with_cache(prompts, return_type="loss", loss_per_token=True)

def deactivate_components_hook(value, hook):
    if "blocks." in hook.name and hook.layer() != 0 and ("attn" in hook.name or "mlp" in hook.name):
        value = ablated_cache[hook.name]
        print(hook.name)
    return value   
deactivate_components_hooks = [(lambda pattern : True, deactivate_components_hook)]

with model.hooks(fwd_hooks=deactivate_components_hooks):
    activated_direct_effect_loss, _ = model.run_with_cache(prompts, return_type="loss", loss_per_token=True)

print((activated_direct_effect_loss - ablated_loss).mean(0)[-1])

# Not a skip bigram

blocks.1.attn.hook_q
blocks.1.attn.hook_k
blocks.1.attn.hook_v
blocks.1.attn.hook_rot_q
blocks.1.attn.hook_rot_k
blocks.1.attn.hook_attn_scores
blocks.1.attn.hook_pattern
blocks.1.attn.hook_z
blocks.1.attn.hook_result
blocks.1.hook_attn_out
blocks.1.mlp.hook_pre
blocks.1.mlp.hook_post
blocks.1.hook_mlp_out
blocks.2.attn.hook_q
blocks.2.attn.hook_k
blocks.2.attn.hook_v
blocks.2.attn.hook_rot_q
blocks.2.attn.hook_rot_k
blocks.2.attn.hook_attn_scores
blocks.2.attn.hook_pattern
blocks.2.attn.hook_z
blocks.2.attn.hook_result
blocks.2.hook_attn_out
blocks.2.mlp.hook_pre
blocks.2.mlp.hook_post
blocks.2.hook_mlp_out
blocks.3.attn.hook_q
blocks.3.attn.hook_k
blocks.3.attn.hook_v
blocks.3.attn.hook_rot_q
blocks.3.attn.hook_rot_k
blocks.3.attn.hook_attn_scores
blocks.3.attn.hook_pattern
blocks.3.attn.hook_z
blocks.3.attn.hook_result
blocks.3.hook_attn_out
blocks.3.mlp.hook_pre
blocks.3.mlp.hook_post
blocks.3.hook_mlp_out
blocks.4.attn.hook_q
blocks.4.attn.hook_k
blocks.4.attn.hook_v
blocks.4.attn

In [54]:
original_loss, original_cache = model.run_with_cache(prompts, return_type="loss", loss_per_token=True)
with model.hooks([(l0_heads, ablate_l2h7_hook)]):
    ablated_loss, ablated_cache = model.run_with_cache(prompts, return_type="loss", loss_per_token=True)

def deactivate_components_hook(value, hook):
    value = ablated_cache[hook.name]
    return value  

def activate_components_hook(value, hook):
    value = original_cache[hook.name]
    return value

components = []
for component_prefix in ["mlp.hook_post", "hook_attn_out"]:
    for block in range(6):
        components.append(f"blocks.{block}.{component_prefix}")

print("Losses when deactivating components and not removing their indirect effects")
total_effects = []
for component in components:
    deactivate_components_hooks = [(component, deactivate_components_hook)]
    with model.hooks(fwd_hooks=deactivate_components_hooks):
        activated_direct_effect_loss, _ = model.run_with_cache(prompts, return_type="loss", loss_per_token=True)
        total_effects.append((activated_direct_effect_loss - original_loss).mean(0)[-1])

for component, total_effect in zip(components, total_effects):
    if total_effect > 0.5:
        print(component, total_effect)


print("Losses when deactivating components and removing their indirect effects")
direct_effects = []
for component in components:
    deactivate_components_hooks = [(component, deactivate_components_hook)]
    activate_future_components_hooks = [(future_component, activate_components_hook) for future_component in components if future_component != component]
    with model.hooks(fwd_hooks=deactivate_components_hooks+activate_future_components_hooks):
        activated_direct_effect_loss, _ = model.run_with_cache(prompts, return_type="loss", loss_per_token=True)
        direct_effects.append((activated_direct_effect_loss - original_loss).mean(0)[-1])

for component, direct_effect in zip(components, direct_effects):
    if direct_effect > 0.5:
        print(component, direct_effect)



Losses when deactivating components and not removing their indirect effects
blocks.1.mlp.hook_post tensor(3.0599, device='cuda:0')
blocks.2.mlp.hook_post tensor(0.7854, device='cuda:0')
blocks.5.mlp.hook_post tensor(1.7032, device='cuda:0')
blocks.0.hook_attn_out tensor(3.0581, device='cuda:0')
Losses when deactivating components and removing their indirect effects
blocks.1.mlp.hook_post tensor(1.3062, device='cuda:0')
blocks.2.mlp.hook_post tensor(1.6621, device='cuda:0')
blocks.5.mlp.hook_post tensor(1.7032, device='cuda:0')
